### Linear Regression Example

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
# Load training data
#training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")
training=spark.read.format('libsvm').option("numFeatures","10").load('sample_linear_regression_data.txt', header=True)

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [8]:
# Fit the model
lrModel = lr.fit(training)

In [9]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept:0.14228558260358093


In [10]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

In [11]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


#### Train/Test Splits

In [14]:
all_data = spark.read.format("libsvm").option("numFeatures","10").load("sample_linear_regression_data.txt", header=True)

In [15]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
+----------

In [17]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|
|-15.348871155379253|(10,[0,1,2,3,4,5,...|
|-15.334767479922341|(10,[0,1,2,3,4,5,...|
|-11.904986902675114|(10,[0,1,2,3,4,5,...|
| -11.87816749996684|(10,[0,1,2,3,4,5,...|
|-11.857350365429426|(10,[0,1,2,3,4,5,...|
|-11.615775265015627|(10,[0,1,2,3,4,5,...|
|-10.945919657782932|(10,[0,1,2,3,4,5,...|
|-10.288657252388708|(10,[0,1,2,3,4,5,...|
| -9.754451457291598|(10,[0,1,2,3,4,5,...|
|  -9.42898793151394|(10,[0,1,2,3,4,5,...|
| -9.196003366226202|(10,[0,1,2,3,4,5,...|
| -8.772667465932606|(10,[0,1,2,3,4,5,...|
| -8.680225911784335|(10,[0,1,2,3,4,5,...|
|  -8.24622879369294|(10,[0,1,2,3,4,5,...|
+----------

In [18]:
unlabeled_data = test_data.select('features')

In [19]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [20]:
correct_model = lr.fit(train_data)

In [21]:
test_results = correct_model.evaluate(test_data)

In [22]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-19.943817141150305|
|-21.112864698891535|
|-17.328951019232356|
|-15.763115626720026|
|-14.948925792799937|
|-17.210497619686134|
| -12.30144644977995|
|-16.904383891158496|
| -12.05840506287443|
|-12.750870285824204|
| -11.50980486138932|
|-13.551595492631112|
|   -9.0939034780561|
| -9.240451260797728|
|-5.9905605610341475|
| -4.797656332826048|
| -9.227381853655999|
|-7.9411158434711595|
|  -8.81969233153704|
| -4.663562149507811|
+-------------------+
only showing top 20 rows

RMSE: 9.492202431572933


In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-2.8936432757690373|
|(10,[0,1,2,3,4,5,...| 1.3301019092769975|
|(10,[0,1,2,3,4,5,...|0.00223028655640764|
|(10,[0,1,2,3,4,5,...|-0.9559812068850624|
|(10,[0,1,2,3,4,5,...|-1.1367332482215524|
|(10,[0,1,2,3,4,5,...| 1.4784093474468878|
|(10,[0,1,2,3,4,5,...|-3.0474247055993025|
|(10,[0,1,2,3,4,5,...|  1.569616411236155|
|(10,[0,1,2,3,4,5,...|0.15341816019931537|
|(10,[0,1,2,3,4,5,...| 0.8727027858573639|
|(10,[0,1,2,3,4,5,...|-0.3475455040401067|
|(10,[0,1,2,3,4,5,...| 1.9358202276154859|
|(10,[0,1,2,3,4,5,...| -1.852016179726831|
|(10,[0,1,2,3,4,5,...| -1.048205991590979|
|(10,[0,1,2,3,4,5,...|-3.7638908962574504|
|(10,[0,1,2,3,4,5,...|-4.6313315986878925|
|(10,[0,1,2,3,4,5,...|0.03137848742979571|
|(10,[0,1,2,3,4,5,...|-0.8315516224614465|
|(10,[0,1,2,3,4,5,...| 0.1394664197527049|
|(10,[0,1,2,3,4,5,...|-3.5826666441851276|
+----------